# Tutorial: Gli strumenti di base per il deep learning privato

Benvenuto! In questo primo tutorial cercheremo di spiegarti come funziona PySyft. PySyft nasce come soluzione alle problematiche di privacy e sicurezza dei dati nei sistemi di machine learning, fornendo strumenti al supporto della decentralizzazione dei modelli e la privatizzazione dei dati. Questa serie di notebook si configura come una guida passo dopo  passo per venire a conoscenza di nuovi sistemi e strumenti al servizio del deep learning su dati, modelli encriptati o privati in maniera decentralizzata. 



**Scopo:** In questi tutorial, non ci limiteremo esclusivamente a trattare in maniera teorica tecniche di decentralizzazione o crittografia dei dati, ma utilizzeremo PySyft per decentralizzare l'intero ecosistema attorno ai dati. Ciò comprende sia i database in cui i dati vengono archiviati, sia i modelli di reti neurali impiegate. Man mano che verranno implementate nuove funzionalità per PySyft, questi notebook verranno estesi con nuovi tutorial per spiegarne il loro funzionamento. 

Autore:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

Traduzione ad opera di:
- Daniele Gadler - Twitter [@DanieleGadler](https://twitter.com/DanieleGadler)
- Andrea Provino 

## Struttura:

- Parte 1: Gli strumenti di base per il deep learning privato


## Perchè dovresti seguire questo tutorial?

**1) Vantaggio competitivo nella ricerca di un lavoro**. Negli ultimi 20 anni, lo sviluppo tecnologico ha prodotto una mole di dati impressionante, riducendo progressivamente il tempo di raddoppio: quello necessario a produrre una quantità di dati pari a quella creata dall’inizio della storia umana al momento della misurazione.

Le nuove regolazioni sui dati, come il [GDPR](https://eugdpr.org/), hanno introdotto regolamenti sempre più stringenti sul passaggio e sulla manipolazione delle informazioni degli utent, motivo per cui oggigiorno le aziende devono fare particolare  attenzione all'utilizzo e all'analisi dei dati utente.

Queste regolamentazioni impongono delle grosse limitazioni ai data scientist, i queli si trovano a dover lavorare su piccoli dataset contenenti poche informazioni utili di alto valore. Apprendendo strumenti privacy-preserving per il deep learning, come PySyft, puoi restare anche tu al passo anche con i tempi e ottenerne un vantaggio per la tua carriera.

**2) Opportunità imprenditoriali**.  Esistono moltissimi problemi sociali che potrebbero essere risolti o attenuati sfruttando tecnologie di deep learning. Al momento, il principale ostacolo per la risoluzione di questi problemi è l’assenza o l'impossibilità di accedere ai dati in conformità con le regolazioni esistenti. Difatti, i sistemi di machine learning hanno bisogno di accedere a molte informazioni sensibili e private su persone con problemi di salute, la cui privacy andrebbe preservata ad ogni costo.

Imparare a utilizzare sistemi per il deep learning privato equivale quindi a fornire nuove opportunità di business alle startup, altrimenti non realizzabili con sistemi di machine learning tradizionali. 


**3) Bene Sociale** - Il deep learning può essere usato per risolvere una grande varietà di problemi del  mondo reale. Ora, creare sistemi di Deep Learning basandosi sulle informazioni personali, significa creare sistemi di Deep Learning *per le persone*.

Imparare ad implementare il deep learning su dati che non si posseggono è molto più che un semplice trampolino per la propria carriera o un' opportunità imprenditoriale: è la possibilità di risolvere alcuni dei problemi più personali e importanti nella vita delle persone - e farlo su una vasta scala.


## Come posso supportare il progetto "PySyft"?

- Dai una stella a PySyft PySyft su GitHub! - [https://github.com/OpenMined/PySyft](https://github.com/OpenMined/PySyft)
- Realizza un video Youtube in cui spieghi il funzionamento di questo notebook!


... ok ... cominciamo!

# Parte - 1: Prerequisiti

- Conoscenza di PyTorch - altrimenti, consulta il course al seguente indirizzo http://fast.ai
- Leggi il paper di PySyft https://arxiv.org/pdf/1811.04017.pdf! Questo paper ti fornirà un'idea generale sull'architettura di PySyft, il che ti aiuterà a comprendere meglio il funzionamento di PySyft.

# Parte 0: Setup

Per cominciare, assicurati di avere le dependencies di PySyft installate. Le istruzioni di installazione sono contenute le file README.MD di PySyft. Per la maggior parte delle persone questo è:

- Installa Python 3.6+
- Installa PyTorch 1.3
- Esegui un clone della repository di PySyft (git clone https://github.com/OpenMined/PySyft.git)
- cd PySyft
- pip install -r pip-dep/requirements.txt
- pip install -r pip-dep/requirements_udacity.txt
- python setup.py install udacity
- python setup.py test

Se riscontri un errore nella procedura di installazione (o se un test case fallisce) - innanzitutto consulta il [README](https://github.com/OpenMined/PySyft.git) per aiuto con l'installazione ed eventualmente apri un Issue su Github. In alternative, prova a sentire se qualcuno nel canale #beginner di Slack ti può dare una mano! [slack.openmined.org](http://slack.openmined.org/)

In [ ]:
#Esegui questa cella per verificare se PySyft è stato installato correttamente
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

Se sei riuscito ad eseguire questa cella con successo, PySyft è stato installato correttamente! Complimenti!

# Parte 1: Gli strumenti di base per Data Science privata e decentralizzata


Quindi, ti starai chiedendo  - Come facciamo ad eseguire il training di modelli su dati a cui non abbiamo accesso?

Be, la risposta è in realtà molto semplice. Se sei abituato a lavorare in PyTorch, allora sei abitiato a lavorare con oggetti torch.Tensor, come i seguenti:

In [ ]:
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)

Ovviamente, è importante usare questi "eleganti" tensori, ma ciò richiede che i dati risiedano sulla tua macchina locale. Ed è qui che comincia il nostro viaggio.


# Section 1.1 - Sending Tensors to Bob's Machine

Mentre normalmente eseguiremo operazioni di machine learning sulla macchine dove i dati risiedono, ora vorremo eseguire questo tipo di computazioni su un' **altra** macchina. Nello specifico, non possiamo più presupporre che i dati risiedano sulla nostra macchina locale. 

Pertanto, invece di usare tensori Torch, ora lavoreremo con **puntatori** a tensori. Per prima cosa, creiamo una macchina fittizia, appartenente ad una persona fittizia, che chiameremo bob.

In [ ]:
bob = sy.VirtualWorker(hook, id="bob")

Supponiamo che la macchine di Bob sia su un altro pianeta, volendo anche su Marte! But, al momento la sua macchina è vuota. Ora creeremo dei dati da mandare a Bob e cercheremo di capire come funzionano i puntatoroi!

Let's say Bob's machine is on another planet - perhaps on Mars! But, at the moment the machine is empty. Let's create some data so that we can send it to Bob and learn about pointers!

In [ ]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

E ora - mandiamo i nostri tensori a Bob!!

In [ ]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [ ]:
x_ptr

BOOM! Ora Bob ha due tensori! Non mi credi? Guarda te stesso!!

In [ ]:
bob._objects

In [ ]:
z = x_ptr + x_ptr

In [ ]:
z

In [ ]:
bob._objects

Ora fai attenzione. Quando abbiamo invocato il comando `x.send(bob)`, questo ha restituito un nuovo oggetto che abbiamo chiamato `x_ptr`. Questo è il nostro primo *puntatore* a un tensore. I puntatori a tensori non contengono dati loro stesso, ma contengono semplicemente dei metadati riguardo ad un tensore, con dati immagazzinati su un'altra macchina. Lo scopo di questi tensori è di fornirci un'API intuitiva per dire all'altra macchina di eseguire delle funzioni usando questo tensore. Diamo un'occhiata al tipo di metadati contenuti nei puntatori.

In [ ]:
x_ptr

Ecco i metadati contenuti in un puntatore!

Vi sono due attributi specifici dei puntatori:

- `x_ptr.location : bob`, la location rappresenta un riferimento al "luogo" a cui il puntatore punta
- `x_ptr.id_at_location : <random integer>`, l'id dove il tensore è immagazzinato. 

I metadati vengono stampati nel seguente formato: `<id_at_location>@<location>`

I metadati contengono altri due attributi generici: 
- `x_ptr.id : <random integer>`, l'id del nostro puntatore al tensore, assegnato in maniera causale
- `x_ptr.owner : "me"`, il worker che possiede il puntatore al tensore, in tal caso il worker locale chiamato "me"


In [ ]:
x_ptr.location

In [ ]:
bob

In [ ]:
bob == x_ptr.location

In [ ]:
x_ptr.id_at_location

In [ ]:
x_ptr.owner

Ti starai forse chiedendo perchè il worker locale che possiede il puntatore è anche un VirtualWorker, nonostante non lo avessimo creato.

La cosa interessante è che, mentre noi avevamo un oggetto VirtualWorker per bob, noi (di default) abbiamo un Virtual Worker anche per noi stessi. Questo worker viene creato automaticamente quando invochiamo il comando `hook = sy.TorchHook()`, cosicchè tu non debba crearlo te stesso. 

In [ ]:
me = sy.local_worker
me

In [ ]:
me == x_ptr.owner

Finalmente, nella stessa maniera in cui possiamo invocare .send() su un tensore, possiamo invocare .get() su un puntatore ad un tensore per "estrarre" il tensore originale dal puntatore.

In [ ]:
x_ptr

In [ ]:
x_ptr.get()

In [ ]:
y_ptr

In [ ]:
y_ptr.get()

In [ ]:
z.get()

In [ ]:
bob._objects

E come puoi vedere, Bob non ha più nessun tensore!!! Tutti i tensori sono ritornati sulla nostra macchina locale! 

# Sezione 1.2 - Utilizzo di Puntatori a Tensori

Quindi, abbiamo ormai capito come mandare e ricevere tensori da Bob, ma sicuramente questo non è Deep Learning! Vorremmo riuscire ad eseguire _operazioni_ su tensori remoti, proprio come eseguimo operazioni su tensori locali. Per nostra fortuna, i puntatori ai tensori rendono questo processo piuttosto facile! Puoi semplicemente usare puntatori a tensori, nella stessa maniera in cui useresti tensori PyTorch normali. 

In [ ]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [ ]:
z = x + y

In [ ]:
z

E voilà!

Dietro questa banale operazione, in realtà è successo qualcosa di molto potente! Invece di eseguire un'operazione di addizione localmente, un comando è stato serializzato e mandato a Bob; Bob ha quindi eseguito la computazione, creato un tensore z, e ci ha quindi restituito un puntatore a z.

Se invochiamo .get() sul puntatore, il risultato ci verrà quindi restituito sulla nostra macchina locale!

In [ ]:
z.get()

### Funzioni Torch

Questa API è stata estesa a tutte le funzioni di PyTorch!!!

In [ ]:
x

In [ ]:
y

In [ ]:
z = torch.add(x,y)
z

In [ ]:
z.get()

### Variabili (inclusa la backpropagation!)

In [ ]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

In [ ]:
z = (x + y).sum()

In [ ]:
z.backward()

In [ ]:
x = x.get()

In [ ]:
x

In [ ]:
x.grad

Quindi, come puoi vedere, la API di PySyft è piuttosto flessibile e può essere utilizzata per eseguire quasi qualsiasi operazione che eseguiresti in PyTorch su *dati remoti*. Queste funzionalità pongeono le fondamenta per l'implementazione di protocolli privacy-preserving più avanzati, tra cui Federated Learning, Secure Multi-Party Computation, e Differential Privacy!

# Congratulazioni!!! - Unisciti alla Community!

Congratulazioni per aver completato questo notebook tutorial! Se ti è piaciuto e vorresti unirti al movimento per la tutela della privacy, la proprietà decentralizzata dell'IA, eccoti alcune possibilità:

### Dai una stella a PySyft su GitHub

Il modo più semplice per supportare la nostra community è dando una stella alle repository Github di OpenMined! Questo aiuta ad aumentare la consapevolezza e l'attenzione verso i  fantastici strumenti che stiamo plasmando.


- [Dai una stella a PySyft](https://github.com/OpenMined/PySyft)

### Unisciti a Slack!

Puoi tenerti aggiornato sugli ultimi progressi unendoti alla nostra comunity! Puoi farlo compilando il modulo all'indirizzo [http://slack.openmined.org](http://slack.openmined.org)

### Unisciti ad un progetto di codice!

Il modo migliore per contribuire alla nostra community è diventare un contributor di codice! In qualsiasi momento puoi andare sugli pagina degli Issues di PySyft, e filtrare per "Progetti". In questa maniera, visualizzerai tutti i ticket di alto livello, fornendoti un'idea dei progetti a cui potresti unirti! 

Se non vuoi unirti ad un progetto, ma vorresti comunque contribuire con del codice, puoi semplicemente cercare dei mini-progetti "Una-Tantum", caratterizzati dal tag "good first issue".

- [Progetti PySyft](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Ticket 'Good First Issue'](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

Se non hai tempo di contribuire al nostro codice, ma vorresti comunque dare una mano, puoi diventare un sostenitore del progetto PySyft sul nostro Open Collective. Tutte le donazioni ricevute verranno devolute al web hosting and ad altre spese della community, tipo hackathons e incontri di PySyft.

[OpenMined's Open Collective Page](https://opencollective.com/openmined)